In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

In [3]:
df_Train=pd.read_csv('Train_New.csv')
df_Test=pd.read_csv('Test_New.csv')

In [4]:
df_Train.drop('PC1', axis=1, inplace=True)
df_Train.drop('PC2', axis=1, inplace=True)
df_Train.drop('PC3', axis=1, inplace=True)
df_Train.drop('PC4', axis=1, inplace=True)
df_Train.drop('PC5', axis=1, inplace=True)
df_Train.drop('Meals', axis=1, inplace=True)

In [5]:
df_Test.drop('PC1', axis=1, inplace=True)
df_Test.drop('PC2', axis=1, inplace=True)
df_Test.drop('PC3', axis=1, inplace=True)
df_Test.drop('PC4', axis=1, inplace=True)
df_Test.drop('PC5', axis=1, inplace=True)
df_Test.drop('Meals', axis=1, inplace=True)

In [6]:
X=df_Train.drop(['Per Person Price'],axis=1)
y=df_Train[['Per Person Price']]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.18,random_state=33)

In [10]:
import optuna
from math import sqrt
import lightgbm as lgb

In [11]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [12]:
from sklearn.metrics import mean_squared_log_error

In [13]:
def objective(trial):    
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 32, 612),
            'max_bin': trial.suggest_int('max_bin', 400, 1000),
            'objective': 'regression',
            'metric': 'RMSE',
            'learning_rate': trial.suggest_float('learning_rate',0.0155,0.1),
            'random_state' : 33,
            'max_depth': trial.suggest_int('max_depth', 4, 16),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 21),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 9),
            'min_child_samples': trial.suggest_int('min_child_samples', 3, 100),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
            'early_stopping_rounds': 10
            }
    
    model = lgb.train(params, lgb_train, valid_sets=[lgb_valid], verbose_eval=100, num_boost_round=800)
    y_pred  = model.predict(X_test)
    score = np.sqrt(mean_squared_log_error(y_test, y_pred))
    print(f"RMSE = {score}")
    return score

In [14]:
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 50)

[I 2021-04-03 18:19:07,876] A new study created in memory with name: no-name-951ae4c8-7c4c-436c-8007-47777c3b7c16
D:\anaconda\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds
[100]	valid_0's rmse: 7958.92
[200]	valid_0's rmse: 7127.13
[300]	valid_0's rmse: 6788.44
[400]	valid_0's rmse: 6577.91
[500]	valid_0's rmse: 6436.57
[600]	valid_0's rmse: 6340.82
[700]	valid_0's rmse: 6255.2
[800]	valid_0's rmse: 6200.04
Did not meet early stopping. Best iteration is:
[800]	valid_0's rmse: 6200.04


[I 2021-04-03 18:19:10,462] Trial 0 finished with value: 0.24222580151246184 and parameters: {'num_leaves': 446, 'max_bin': 783, 'learning_rate': 0.022994612098661667, 'max_depth': 5, 'min_child_weight': 2, 'feature_fraction': 0.7581941306350666, 'bagging_fraction': 0.5749628008883861, 'bagging_freq': 9, 'min_child_samples': 76, 'lambda_l1': 5.234048452818184e-08, 'lambda_l2': 0.00020400398965834046}. Best is trial 0 with value: 0.24222580151246184.


RMSE = 0.24222580151246184
Training until validation scores don't improve for 10 rounds


D:\anaconda\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

RMSE = 0.2074612890683057
Training until validation scores don't improve for 10 rounds


D:\anaconda\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	valid_0's rmse: 5538.43
[200]	valid_0's rmse: 5268.61
Early stopping, best iteration is:
[234]	valid_0's rmse: 5232.02


[I 2021-04-03 18:22:49,775] Trial 49 finished with value: 0.19679933819128076 and parameters: {'num_leaves': 516, 'max_bin': 748, 'learning_rate': 0.033347200309025216, 'max_depth': 12, 'min_child_weight': 10, 'feature_fraction': 0.669439278344041, 'bagging_fraction': 0.8657135349871047, 'bagging_freq': 9, 'min_child_samples': 3, 'lambda_l1': 0.0052941295772233965, 'lambda_l2': 0.007927337055989832}. Best is trial 32 with value: 0.19353207092479338.


RMSE = 0.19679933819128076


In [15]:
print(f"Trials : {len(study.trials)}")
print(f"Best Trial : {study.best_trial}")
best = study.best_trial
for i, j in best.params.items():
    print(f"{i} : {j}")
print(f"{best.value}")

Trials : 50
Best Trial : FrozenTrial(number=32, values=[0.19353207092479338], datetime_start=datetime.datetime(2021, 4, 3, 18, 20, 59, 401252), datetime_complete=datetime.datetime(2021, 4, 3, 18, 21, 6, 769964), params={'num_leaves': 348, 'max_bin': 617, 'learning_rate': 0.04611514228609913, 'max_depth': 15, 'min_child_weight': 5, 'feature_fraction': 0.6573572112800595, 'bagging_fraction': 0.8274438256193789, 'bagging_freq': 5, 'min_child_samples': 3, 'lambda_l1': 0.00028253226619165807, 'lambda_l2': 1.854341459205825e-05}, distributions={'num_leaves': IntUniformDistribution(high=612, low=32, step=1), 'max_bin': IntUniformDistribution(high=1000, low=400, step=1), 'learning_rate': UniformDistribution(high=0.1, low=0.0155), 'max_depth': IntUniformDistribution(high=16, low=4, step=1), 'min_child_weight': IntUniformDistribution(high=21, low=1, step=1), 'feature_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_freq': Int

In [152]:
import lightgbm as lgb
clf = lgb.LGBMRegressor(num_leaves=348, max_bin=617, learning_rate=0.09411514228609913, max_depth=15,
min_child_weight=5, feature_fraction=0.6573572112800595, bagging_fraction=0.8274438256193789, bagging_freq=5,
min_child_samples=3, lambda_l1=0.00028253226619165807, lambda_l2=1.854341459205825e-05)
clf.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.8274438256193789, bagging_freq=5,
              feature_fraction=0.6573572112800595,
              lambda_l1=0.00028253226619165807, lambda_l2=1.854341459205825e-05,
              learning_rate=0.09411514228609913, max_bin=617, max_depth=15,
              min_child_samples=3, min_child_weight=5, num_leaves=348)

In [153]:
pred = clf.predict(X_test)

from sklearn.metrics import r2_score
print(r2_score(y_test, pred))

0.8055185774683529


In [154]:
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_test, pred))

0.19601039271733378

In [150]:
prediction = clf.predict(df_Test)
prediction

array([19394.14771293, 11602.57166886,  5000.37499447, ...,
       19853.97568822, 21825.99013654, 11606.93019233])

In [151]:
output = pd.DataFrame({'Per Person Price': prediction})
output.to_csv('WorkathonStarterSubmission_04_04_2.csv', index=False)